In [1]:
!pip -q install kaggle
import os, json, pathlib

from pathlib import Path
kaggle_src = Path('/content/kaggle.json')
assert kaggle_src.exists(), "Upload kaggle.json to /content first"
!mkdir -p ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Dowload dataset from Kaggle and unzip

In [2]:
dataset = 'patrickzel/flight-delay-and-cancellation-dataset-2019-2023'
!kaggle datasets download -d $dataset --unzip -p /content

Dataset URL: https://www.kaggle.com/datasets/patrickzel/flight-delay-and-cancellation-dataset-2019-2023
License(s): other
 94% 132M/140M [00:00<00:00, 1.38GB/s]
100% 140M/140M [00:00<00:00, 1.30GB/s]


(0) **High-level plan**



1.   Inspect files & load CSV into a DataFrame (verify column names).
2.   Clean the data and create the target: is_delayed = ARR_DELAY > 5 (drop cancelled flights).
3.   Exploratory Data Analysis (EDA): class balance, missingness, patterns by hour/day/airport/carrier.
4.   Feature engineering: extract hour, weekday, encode categorical features (carrier, origin, dest, etc.).
5.   Choose modeling strategy (time-based split → baseline model → stronger models). Time matters here: this is time series / sequential flight records — train on older flights, test on later flights.
MDPI
6.   Evaluate with appropriate metrics (precision/recall/F1, ROC AUC) and check feature importance.
7.   Save the model and next steps (more data like weather, better encodings, SHAP explanations, production).

(1) **Start in Colab — find & load the file**

Run this first to find your CSV(s) and load into pandas. (This does not download anything to your laptop — it runs inside Colab.)

**Why**: verify you actually have the dataset in Colab and see exact column names. Column names in different releases can vary a little — always inspect them first.

In [3]:
# 1) locate files in current Colab environment
import glob, os
print("Files in /content:")
print(os.listdir('/content')[:50])

# find CSVs that look like flights
candidates = [f for f in glob.glob('/content/**/*.csv', recursive=True)]
print("CSV candidates:", candidates[:20])

# Adjust filename if needed. Example:
csv_path = candidates[0]   # change index if needed
print("Using:", csv_path)

import pandas as pd
df = pd.read_csv(csv_path, low_memory=False)   # low_memory to avoid dtype warnings
print(df.shape)
print(df.columns.tolist()[:80])  # first N columns
df.head()


Files in /content:
['.config', 'dictionary.html', 'kaggle.json', 'flights_sample_3m.csv', 'sample_data']
CSV candidates: ['/content/flights_sample_3m.csv', '/content/sample_data/mnist_test.csv', '/content/sample_data/california_housing_test.csv', '/content/sample_data/mnist_train_small.csv', '/content/sample_data/california_housing_train.csv']
Using: /content/flights_sample_3m.csv
(3000000, 32)
['FL_DATE', 'AIRLINE', 'AIRLINE_DOT', 'AIRLINE_CODE', 'DOT_CODE', 'FL_NUMBER', 'ORIGIN', 'ORIGIN_CITY', 'DEST', 'DEST_CITY', 'CRS_DEP_TIME', 'DEP_TIME', 'DEP_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'WHEELS_ON', 'TAXI_IN', 'CRS_ARR_TIME', 'ARR_TIME', 'ARR_DELAY', 'CANCELLED', 'CANCELLATION_CODE', 'DIVERTED', 'CRS_ELAPSED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE', 'DELAY_DUE_CARRIER', 'DELAY_DUE_WEATHER', 'DELAY_DUE_NAS', 'DELAY_DUE_SECURITY', 'DELAY_DUE_LATE_AIRCRAFT']


,FL_DATE,AIRLINE,AIRLINE_DOT,AIRLINE_CODE,DOT_CODE,FL_NUMBER,ORIGIN,ORIGIN_CITY,DEST,DEST_CITY,...,DIVERTED,CRS_ELAPSED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,DELAY_DUE_CARRIER,DELAY_DUE_WEATHER,DELAY_DUE_NAS,DELAY_DUE_SECURITY,DELAY_DUE_LATE_AIRCRAFT
0,2019-01-09,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,1562,FLL,"Fort Lauderdale, FL",EWR,"Newark, NJ",...,0.0,186.0,176.0,153.0,1065.0,NaN,NaN,NaN,NaN,NaN
1,2022-11-19,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,1149,MSP,"Minneapolis, MN",SEA,"Seattle, WA",...,0.0,235.0,236.0,189.0,1399.0,NaN,NaN,NaN,NaN,NaN
2,2022-07-22,United Air Lines Inc.,United Air Lines Inc.: UA,UA,19977,459,DEN,"Denver, CO",MSP,"Minneapolis, MN",...,0.0,118.0,112.0,87.0,680.0,NaN,NaN,NaN,NaN,NaN
3,2023-03-06,Delta Air Lines Inc.,Delta Air Lines Inc.: DL,DL,19790,2295,MSP,"Minneapolis, MN",SFO,"San Francisco, CA",...,0.0,260.0,285.0,249.0,1589.0,0.0,0.0,24.0,0.0,0.0
4,2020-02-23,Spirit Air Lines,Spirit Air Lines: NK,NK,20416,407,MCO,"Orlando, FL",DFW,"Dallas/Fort Worth, TX",...,0.0,181.0,182.0,153.0,985.0,NaN,NaN,NaN,NaN,NaN


(2) **Quick cleaning & create the target is_delayed**

We’ll: normalize column names to uppercase, drop cancelled flights, and create IS_DELAYED = ARR_DELAY > 5.

Why drop cancelled flights? Cancellations don’t have meaningful arrival delay values (NaNs). You can treat cancellations separately if you want multi-class prediction later, but for a simple delayed/not-delayed classifier it’s cleaner to exclude them. Also, >5 minutes is your chosen threshold — common practice in flight analysis.

In [4]:
# Standardize column names (easier to refer to)
df.columns = [c.strip().upper() for c in df.columns]

# Common column names to check:
print('Important columns present?',
      'ARR_DELAY' in df.columns,
      'DEP_DELAY' in df.columns,
      'CANCELLED' in df.columns,
      'FL_DATE' in df.columns)

# Remove cancelled flights (they won't have reliable arrival delay)
if 'CANCELLED' in df.columns:
    df = df[df['CANCELLED'] == 0].copy()

# Remove rows with no ARR_DELAY
df = df[df['ARR_DELAY'].notna()].copy()

# Create binary target: delayed if arrival delay > 5 minutes
df['IS_DELAYED'] = (df['ARR_DELAY'] > 5).astype(int)

# Quick reality check
print("Rows, columns:", df.shape)
print("Delay distribution (counts and fraction):")
print(df['IS_DELAYED'].value_counts(), df['IS_DELAYED'].value_counts(normalize=True))


Important columns present? True True True True
Rows, columns: (2913802, 33)
Delay distribution (counts and fraction):
IS_DELAYED
0    2146479
1     767323
Name: count, dtype: int64 IS_DELAYED
0    0.736659
1    0.263341
Name: proportion, dtype: float64


(3) **Exploratory Data Analysis (EDA) — what to look at**

Run these to understand missing data, distribution across carriers, hours, airports, and to check class imbalance:

**Why**: EDA tells you where the signal may be (e.g., some hours or airports have more delays), reveals missing values you must handle, and shows class imbalance — if 80% flights are on-time, accuracy alone is meaningless.

In [5]:
# Missing values by column (top)
missing = df.isna().sum().sort_values(ascending=False).head(30)
print(missing)

# Delay distribution by hour / day / carrier
def safe_show(col):
    if col in df.columns:
        print(f"\n=== {col} distribution ===")
        print(df[col].value_counts().head(20))

for col in ['OP_CARRIER','ORIGIN','DEST','MONTH','DAY_OF_WEEK']:
    safe_show(col)

# Create a departure hour column if CRS_DEP_TIME or similar exists
time_cols = [c for c in df.columns if 'DEP' in c and 'CRS' in c or 'DEP_TIME' in c.upper()]
print("Possible time columns:", time_cols[:20])

# Example: create hour from common "CRS_DEP_TIME" (format like 730, 1530)
if 'CRS_DEP_TIME' in df.columns:
    def get_hour(x):
        try:
            x = int(x)
            return x // 100
        except:
            return None
    df['DEP_HOUR'] = df['CRS_DEP_TIME'].apply(get_hour)
    print(df['DEP_HOUR'].value_counts().sort_index().head(24))

# Average arrival delay by hour
if 'DEP_HOUR' in df.columns:
    print(df.groupby('DEP_HOUR')['ARR_DELAY'].median().sort_index().head(24))


CANCELLATION_CODE          2913802
DELAY_DUE_SECURITY         2379939
DELAY_DUE_LATE_AIRCRAFT    2379939
DELAY_DUE_WEATHER          2379939
DELAY_DUE_CARRIER          2379939
DELAY_DUE_NAS              2379939
FL_DATE                          0
AIRLINE                          0
AIRLINE_DOT                      0
DEST_CITY                        0
CRS_DEP_TIME                     0
AIRLINE_CODE                     0
DOT_CODE                         0
FL_NUMBER                        0
ORIGIN                           0
ORIGIN_CITY                      0
DEST                             0
TAXI_IN                          0
WHEELS_ON                        0
WHEELS_OFF                       0
TAXI_OUT                         0
DEP_DELAY                        0
DEP_TIME                         0
ARR_DELAY                        0
CRS_ARR_TIME                     0
ELAPSED_TIME                     0
CRS_ELAPSED_TIME                 0
DIVERTED                         0
CANCELLED           

(4) **Feature engineering — build features the model can use**

Common features that matter for delays:

scheduled departure hour (peak traffic times matter), day_of_week, month (seasonality)

airline/carrier (different performance), origin/destination airports (busy hubs), distance

recent system load / rolling features (advanced: needs sequential data)

Minimal practical transformations (works for large dataset):

**Why these transforms**: hour/day/month capture temporal patterns; carrier and airport capture structural differences; frequency encoding keeps airport information but avoids exploding dimensionality with thousands of one-hot columns (airports can be many). Frequency encoding is simple and stable for tree models and logistic models.

In [6]:
import numpy as np

# 1) FLIGHT DATE -> make sure it's datetime (if present)
if 'FL_DATE' in df.columns:
    df['FL_DATE'] = pd.to_datetime(df['FL_DATE'])

# 2) DEP_HOUR — if not created earlier, try from 'CRS_DEP_TIME' or 'DEP_TIME' fields
if 'DEP_HOUR' not in df.columns:
    if 'CRS_DEP_TIME' in df.columns:
        df['DEP_HOUR'] = df['CRS_DEP_TIME'].astype(float).fillna(0).astype(int) // 100
    elif 'DEP_TIME' in df.columns:
        df['DEP_HOUR'] = df['DEP_TIME'].astype(float).fillna(0).astype(int) // 100
    else:
        # fallback: if no hour available, create dummy
        df['DEP_HOUR'] = 0

# 3) WEEKEND flag
if 'DAY_OF_WEEK' in df.columns:
    df['IS_WEEKEND'] = df['DAY_OF_WEEK'].isin([6,7]).astype(int)

# 4) Frequency encoding for ORIGIN / DEST (cheap, effective)
for c in ['ORIGIN','DEST']:
    if c in df.columns:
        freq = df[c].value_counts(normalize=True)
        df[c + '_FREQ'] = df[c].map(freq)

# 5) Keep a small set of features for baseline
candidate_features = []
for f in ['DEP_HOUR','DAY_OF_WEEK','MONTH','DISTANCE','OP_CARRIER','ORIGIN_FREQ','DEST_FREQ','IS_WEEKEND']:
    if f in df.columns:
        candidate_features.append(f)

print("Baseline features:", candidate_features)
X = df[candidate_features].copy()
y = df['IS_DELAYED'].copy()


Baseline features: ['DEP_HOUR', 'DISTANCE', 'ORIGIN_FREQ', 'DEST_FREQ']


(5) **Train / validation split — use time-based split**

This dataset is time-ordered: if you randomly shuffle, you may leak future patterns. So train on earlier flights and test on later flights — this simulates real prediction.

**Why time-based split?** Because delays change over time (seasonality, airline practices, external events). Training on future rows to predict past rows gives an unrealistic, over-optimistic result.

In [7]:
# If FL_DATE exists, sort by date and split chronologically
if 'FL_DATE' in df.columns:
    df = df.sort_values('FL_DATE').reset_index(drop=True)
    split_idx = int(len(df) * 0.8)   # train 80% (older); test 20% (newer)
    train = df.iloc[:split_idx].copy()
    test  = df.iloc[split_idx:].copy()
else:
    # fallback: random stratified split
    from sklearn.model_selection import train_test_split
    train, test = train_test_split(df, test_size=0.2, random_state=42, stratify=y)

X_train = train[candidate_features]
y_train = train['IS_DELAYED']
X_test  = test[candidate_features]
y_test  = test['IS_DELAYED']

print("Train/test sizes:", X_train.shape, X_test.shape)


Train/test sizes: (2331041, 4) (582761, 4)


(6) **Baseline model: Logistic Regression (fast, interpretable)**

We’ll build a pipeline that imputes missing numeric values, scales numeric features (for LR), and one-hot encodes small categorical features.

**Why Logistic Regression?** It’s fast, interpretable (coefficients), and gives a baseline. If LR does poorly, that tells you there’s nonlinear structure (so tree models will help).

In [9]:
# minimal preprocessing pipeline + logistic regression baseline
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression

num_features = [c for c in candidate_features if df[c].dtype.kind in 'biufc']  # numeric
cat_features = [c for c in candidate_features if c not in num_features]

num_features, cat_features

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([('impute', SimpleImputer(strategy='median')), ('scale', StandardScaler())]), num_features),
        ('cat', Pipeline([('impute', SimpleImputer(strategy='most_frequent')), ('ohe', OneHotEncoder(handle_unknown='ignore', sparse_output=False))]), cat_features),
    ], remainder='passthrough'  # passthrough any other columns if present
)

clf = Pipeline([
    ('pre', preprocessor),
    ('lr', LogisticRegression(max_iter=400, class_weight='balanced'))  # class_weight helps if imbalance exists
])

clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
y_proba = clf.predict_proba(X_test)[:,1]

(7) **Evaluate: accuracy, precision, recall, F1, ROC AUC**

**Why multiple metrics?** If dataset is imbalanced (more on-time flights), accuracy can be misleading — you want precision/recall and ROC AUC to understand real performance. For example, you may care more about identifying delays (recall) or avoiding false alarms (precision).

In [15]:
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, roc_curve

print(classification_report(y_test, y_pred))
print("ROC AUC:", roc_auc_score(y_test, y_proba))

# Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion matrix:\n", cm)


              precision    recall  f1-score   support

           0       0.75      0.58      0.65    397179
           1       0.40      0.59      0.47    185582

    accuracy                           0.58    582761
   macro avg       0.57      0.58      0.56    582761
weighted avg       0.64      0.58      0.60    582761

ROC AUC: 0.608317278691916
Confusion matrix:
 [[230048 167131]
 [ 76142 109440]]


(8) Stronger models: Random Forest / XGBoost (usually much better)

Try tree methods — they handle nonlinearities and categorical interactions better.

In [11]:
# install xgboost if not present
!pip install -q xgboost

from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV

# RandomForest pipeline
from sklearn.pipeline import make_pipeline
rf_pipe = Pipeline([('pre', preprocessor), ('rf', RandomForestClassifier(n_estimators=200, n_jobs=-1, class_weight='balanced'))])

# Quick Random Search (smaller number of parameter combos)
param_dist = {
    'rf__n_estimators': [100, 200],
    'rf__max_depth': [8, 12, None],
    'rf__min_samples_split': [2, 5, 10]
}

rsearch = RandomizedSearchCV(rf_pipe, param_distributions=param_dist, n_iter=6, cv=3, scoring='roc_auc', n_jobs=-1, random_state=42)
rsearch.fit(X_train, y_train)
print("Best score (cv):", rsearch.best_score_)
best_rf = rsearch.best_estimator_


/usr/local/lib/python3.12/dist-packages/joblib/externals/loky/process_executor.py:782: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Best score (cv): 0.6015322471597919


And for XGBoost:

**Why tree models?** They capture nonlinear interactions (e.g., combination of hour + airport) much better than LR. XGBoost often gives state-of-the-art performance on tabular data.

In [14]:
xgb_pipe = Pipeline([('pre', preprocessor), ('xgb', XGBClassifier(use_label_encoder=False, eval_metric='auc'))])
param_dist_xgb = {
    'xgb__n_estimators': [100,200],
    'xgb__max_depth': [4,6,8],
    'xgb__learning_rate': [0.01, 0.1, 0.2]
}
xsearch = RandomizedSearchCV(xgb_pipe, param_distributions=param_dist_xgb, n_iter=6, cv=3, scoring='roc_auc', n_jobs=-1, random_state=42)
xsearch.fit(X_train, y_train)
print("Best XGB cv score:", xsearch.best_score_)
best_xgb = xsearch.best_estimator_


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [17:26:08] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Best XGB cv score: 0.6037444103573985


In [16]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score

def evaluate_model(model, X_train, X_test, y_train, y_test, model_name="Model"):
    # Train the model
    model.fit(X_train, y_train)

    # Predictions
    y_pred = model.predict(X_test)
    y_proba = model.predict_proba(X_test)[:, 1]

    # Metrics
    print(f"\n==== {model_name} ====")
    print(classification_report(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_proba))

    # Confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion matrix:\n", cm)

    return {
        "model": model_name,
        "roc_auc": roc_auc_score(y_test, y_proba),
        "accuracy": (y_pred == y_test).mean()
    }


In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

# Logistic Regression
log_reg = LogisticRegression(max_iter=1000, class_weight="balanced", solver="liblinear")
results_lr = evaluate_model(log_reg, X_train, X_test, y_train, y_test, "Logistic Regression")

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
results_rf = evaluate_model(rf, X_train, X_test, y_train, y_test, "Random Forest")

# Gradient Boosting (XGBoost)
xgb = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    max_depth=6,
    random_state=42,
    scale_pos_weight=1, # tweak for imbalance if needed
    use_label_encoder=False,
    eval_metric="logloss"
)
results_xgb = evaluate_model(xgb, X_train, X_test, y_train, y_test, "XGBoost")



==== Logistic Regression ====
              precision    recall  f1-score   support

           0       0.75      0.58      0.65    397179
           1       0.40      0.59      0.47    185582

    accuracy                           0.58    582761
   macro avg       0.57      0.58      0.56    582761
weighted avg       0.64      0.58      0.60    582761

ROC AUC: 0.6082762459579112
Confusion matrix:
 [[229779 167400]
 [ 76092 109490]]

==== Random Forest ====
              precision    recall  f1-score   support

           0       0.74      0.61      0.67    397179
           1       0.40      0.54      0.46    185582

    accuracy                           0.59    582761
   macro avg       0.57      0.58      0.56    582761
weighted avg       0.63      0.59      0.60    582761

ROC AUC: 0.602903607775146
Confusion matrix:
 [[243555 153624]
 [ 84717 100865]]


/usr/local/lib/python3.12/dist-packages/xgboost/training.py:183: UserWarning: [18:10:02] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



==== XGBoost ====
              precision    recall  f1-score   support

           0       0.68      1.00      0.81    397179
           1       0.54      0.00      0.01    185582

    accuracy                           0.68    582761
   macro avg       0.61      0.50      0.41    582761
weighted avg       0.64      0.68      0.55    582761

ROC AUC: 0.6328317000678593
Confusion matrix:
 [[396531    648]
 [184817    765]]


In [18]:
import pandas as pd

comparison_df = pd.DataFrame([results_lr, results_rf, results_xgb])
print("\n=== Model Comparison ===")
print(comparison_df)



=== Model Comparison ===
                 model   roc_auc  accuracy
0  Logistic Regression  0.608276  0.582175
1        Random Forest  0.602904  0.591014
2              XGBoost  0.632832  0.681748


(9) **Feature importance & interpretability**

For tree models you can extract feature importances and use SHAP for deeper insight.

**Why**: Feature importance tells you which inputs the model used most — helpful to understand whether airport, carrier or time drives predictions.

If you want deeper explanations (per-prediction) use SHAP (it’s very informative for tree models).

In [27]:
# Step 1: Ensure the best pipeline is fitted
# (RandomizedSearchCV already fits it, so best_rf is fitted)
# If needed, you can fit on full training data again:
best_rf.fit(X_train, y_train)

# Step 2: Transform a sample to trigger fitted preprocessor
X_transformed = best_rf.named_steps['pre'].transform(X_train.iloc[:1])

# Step 3: Get the feature names from the ColumnTransformer
# works even if OneHotEncoder is inside a pipeline
def get_feature_names(column_transformer, numeric_features, cat_features):
    feature_names = []

    # numeric columns (passed through)
    feature_names.extend(numeric_features)

    # categorical columns
    ohe = column_transformer.named_transformers_['cat']['ohe']
    feature_names.extend(ohe.get_feature_names_out(cat_features))

    return feature_names

feature_names = get_feature_names(best_rf.named_steps['pre'], num_features, cat_features)

# Step 4: Feature importances
importances = best_rf.named_steps['rf'].feature_importances_
imp_df = pd.DataFrame({'feature': feature_names, 'importance': importances})
imp_df.sort_values('importance', ascending=False).head(20)


NotFittedError: This OneHotEncoder instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [31]:
sample = X_test.iloc[[7000]]   # single row
prob_delay = xsearch.predict_proba(sample)[:,1][0]
print("Predicted probability of delay (>5min):", prob_delay)


Predicted probability of delay (>5min): 0.3641503
